# ETL Validation Notebook — Pharma Supply Chain

## 1. Context and Objective
Validate and prepare production and logistics data for urgent executive reporting.

## 2. Data Loading
- Load raw CSV files
- Initial structure and schema validation

## 3. Data Quality Checks
- Null values
- Duplicate batch IDs
- Negative or inconsistent values
- Date consistency

## 4. Cleaning and Standardization
- Normalize plant and product names
- Handle missing quality issues
- Standardize date formats

## 5. Feature Engineering
- Reject rate
- Cost per unit
- Critical delay flags
- Total lead time

## 6. Validation Summary
- Records removed or corrected
- Remaining data quality risks

## 7. Export for Tableau
- Save clean datasets to /processed


In [6]:
import pandas as pd

## EXTRACT: Data Loading

In [62]:
prod_path  = "../data/raw/production_batches.csv"
dist_path  = "../data/raw/distribution.csv"
plants_path= "../data/raw/plants.csv"

production = pd.read_csv(prod_path)
distribution = pd.read_csv(dist_path)
plants = pd.read_csv(plants_path)


In [ ]:
# Quick inspection
print("production:", production.shape)
print("distribution:", distribution.shape)
print("plants:", plants.shape)

display(production.head(3))
display(distribution.head(3))
display(plants.head(3))


production: (800, 11)
distribution: (1000, 6)
plants: (8, 4)


,batch_id,plant,country,product,production_date,units_produced,units_rejected,production_time_hrs,delay_days,production_cost,quality_issue
0,BCH000000,Plant_F,France,Paracetamol 500mg,2023-11-27,79087,19548,9.3,0,486716.37,Contamination
1,BCH000001,Plant_F,France,Ibuprofen 400mg,2023-12-20,136725,21811,12.7,7,1021781.90,Equipment failure
2,BCH000002,Plant_E,Brazil,Insulin Glargine,2023-07-23,152301,9207,21.5,0,180452.70,NaN


,batch_id,region,planned_delivery,actual_delivery,transport_days,delivery_status
0,BCH000269,East Asia,2023-11-24,2023-11-25,8,On-time
1,BCH000417,South Asia,2023-12-02,2023-12-09,7,Delayed
2,BCH000666,Eastern Europe,2023-07-13,2023-07-13,5,On-time


,plant,country,capacity_units_per_month,plant_type
0,Plant_A,Mexico,380000,Solid
1,Plant_B,USA,420000,Solid
2,Plant_C,Germany,130000,Biological


In [99]:
expected_prod = {
    "batch_id","plant","country","product","production_date",
    "units_produced","units_rejected","production_time_hrs",
    "delay_days","production_cost","quality_issue"
}

expected_dist = {
    "batch_id","region","planned_delivery","actual_delivery",
    "transport_days","delivery_status"
}

expected_plants = {
    "plant","country","capacity_units_per_month","plant_type"
}

missing_prod = expected_prod - set(production.columns)
missing_dist = expected_dist - set(distribution.columns)
missing_plants = expected_plants - set(plants.columns)

print("Missing columns in production:", missing_prod)
print("Missing columns in distribution:", missing_dist)
print("Missing columns in plants:", missing_plants)


print("\n=== NULLS (production) ===")
print(production.isna().sum().sort_values(ascending=False).head(10))

print("\n=== NULLS (distribution) ===")
print(distribution.isna().sum().sort_values(ascending=False).head(10))

print("\n=== NULLS (plants) ===")
print(plants.isna().sum().sort_values(ascending=False).head(10))


Missing columns in production: set()
Missing columns in distribution: set()
Missing columns in plants: set()

=== NULLS (production) ===
quality_issue          264
batch_id                 0
plant                    0
product                  0
country                  0
units_produced           0
units_rejected           0
production_time_hrs      0
production_date          0
delay_days               0
dtype: int64

=== NULLS (distribution) ===
batch_id               0
region                 0
planned_delivery       0
actual_delivery        0
transport_days         0
delivery_status        0
delivery_delay_days    0
critical_delay_ship    0
on_time_flag           0
dtype: int64

=== NULLS (plants) ===
plant                       0
country                     0
capacity_units_per_month    0
plant_type                  0
dtype: int64


### Extract Summary

- All datasets loaded successfully
- No missing critical columns
- No duplicate batch_id detected in production
- Distribution data aligned with production batches
- Data ready for transformation phase


## TRANSFORM — Date convertion

In [ ]:
# Convert dates in production
production["production_date"] = pd.to_datetime(
    production["production_date"],
    errors="coerce"
)

# Convert dates in distribution
distribution["planned_delivery"] = pd.to_datetime(
    distribution["planned_delivery"],
    errors="coerce"
)

distribution["actual_delivery"] = pd.to_datetime(
    distribution["actual_delivery"],
    errors="coerce"
)


In [100]:
print("production_date dtype:", production["production_date"].dtype)
print("planned_delivery dtype:", distribution["planned_delivery"].dtype)
print("actual_delivery dtype:", distribution["actual_delivery"].dtype)

print("\nNull dates after conversion:")
print("production:", production["production_date"].isna().sum())
print("planned_delivery:", distribution["planned_delivery"].isna().sum())
print("actual_delivery:", distribution["actual_delivery"].isna().sum())


production_date dtype: object
planned_delivery dtype: object
actual_delivery dtype: object

Null dates after conversion:
production: 0
planned_delivery: 0
actual_delivery: 0


## Transform — Date Standardization

- All date fields were successfully converted to datetime format
- No invalid or missing dates detected after conversion
- Data is ready for further cleaning and feature engineering


In [ ]:
# Simple normalization function
def normalize_text(series):
    return (
        series
        .astype(str)
        .str.strip()
        .str.replace(r"\s+", " ", regex=True)
    )

# production
production["plant"]   = normalize_text(production["plant"])
production["country"] = normalize_text(production["country"])
production["product"] = normalize_text(production["product"])

# distribution
distribution["region"] = normalize_text(distribution["region"])

# plants
plants["plant"]      = normalize_text(plants["plant"])
plants["country"]    = normalize_text(plants["country"])
plants["plant_type"] = normalize_text(plants["plant_type"])


In [101]:
print("Unique plants (production):")
print(production["plant"].unique())

print("\nUnique countries (production):")
print(production["country"].unique())

print("\nPlant types (plants):")
print(plants["plant_type"].unique())

Unique plants (production):
['Plant_F' 'Plant_E' 'Plant_C' 'Plant_D' 'Plant_A' 'Plant_B']

Unique countries (production):
['France' 'Brazil' 'Germany' 'India' 'Mexico' 'USA']

Plant types (plants):
['Solid' 'Biological']


## Transform — Text Standardization

- Text fields standardized to avoid join and filter inconsistencies
- No records removed during this step
- Data integrity preserved


In [102]:
# Quick counts of numeric issues
issues = {}

issues["units_produced <= 0"] = (production["units_produced"] <= 0).sum()
issues["units_rejected < 0"] = (production["units_rejected"] < 0).sum()
issues["units_rejected > units_produced"] = (
    production["units_rejected"] > production["units_produced"]
).sum()
issues["production_time_hrs <= 0"] = (production["production_time_hrs"] <= 0).sum()
issues["delay_days < 0"] = (production["delay_days"] < 0).sum()
issues["production_cost <= 0"] = (production["production_cost"] <= 0).sum()
issues["transport_days < 0"] = (distribution["transport_days"] < 0).sum()

issues


{'units_produced <= 0': np.int64(0),
 'units_rejected < 0': np.int64(0),
 'units_rejected > units_produced': np.int64(0),
 'production_time_hrs <= 0': np.int64(0),
 'delay_days < 0': np.int64(0),
 'production_cost <= 0': np.int64(0),
 'transport_days < 0': np.int64(0)}

### Transform Summary — Text & Numeric Validation

- Text fields standardized successfully
- No invalid or inconsistent numeric values detected
- Dataset is clean and ready for KPI calculation


## FEATURE ENGINEERING

### reject_rate
What percentage of the batch was rejected due to quality issues?

reject_rate = units_rejected / units_produced


In [70]:
production["reject_rate"] = (
    production["units_rejected"] / production["units_produced"]
)


In [71]:
production["reject_rate"].describe()


count    800.000000
mean       0.124944
std        0.093368
min        0.002870
25%        0.057405
50%        0.098924
75%        0.168468
max        0.560097
Name: reject_rate, dtype: float64

#### KPI: Reject Rate
Calculated as the proportion of rejected units per batch.
Used to assess production quality.

---
“Relevant outliers are observed in the rejection rate, indicating batches with severe quality problems.”

### cost_per_unit
How much does it cost to produce a single unit in each batch?

cost_per_unit = production_cost / units_produced


In [72]:
production["cost_per_unit"] = (
    production["production_cost"] / production["units_produced"]
)


In [73]:
production["cost_per_unit"].describe()


count    800.000000
mean       3.148908
std        2.648207
min        1.080011
25%        1.216126
50%        1.345941
75%        6.360217
max        7.792852
Name: cost_per_unit, dtype: float64

#### KPI: Cost per Unit
Calculated to support cost-efficiency analysis across plants and products.


### Flag — quality_flag

Did this batch have any reported quality issues?

quality_flag = 1 if quality_flag not 1

               0 otherwise

In [74]:
production["quality_flag"] = (
    production["quality_issue"].notna().astype(int)
)


In [75]:
production["quality_flag"].value_counts()


quality_flag
1    536
0    264
Name: count, dtype: int64

### Flag: Quality Issue
Binary indicator for presence of any reported quality issue.


#### critical_quality
Is the rejection level operationally critical?

critical_quality = 1 if reject_rate > 0.05

                   0 otherwise



In [76]:
production["critical_quality"] = (
    (production["reject_rate"] > 0.05).astype(int)
)


In [77]:
production["critical_quality"].value_counts()


critical_quality
1    635
0    165
Name: count, dtype: int64

### Flag: Critical Quality
Identifies batches with reject rate above 5%.


#### critical_delay_prod
Did this batch experience a critical production delay?

critical_delay_prod = 1 if delay_days >= 3

                      0 otherwise


In [78]:
production["critical_delay_prod"] = (
    (production["delay_days"] >= 3).astype(int)
)


In [79]:
production["critical_delay_prod"].value_counts()


critical_delay_prod
0    606
1    194
Name: count, dtype: int64

### Flag: Critical Production Delay
Flags batches delayed by 3 days or more.


#### batch_risk_score
How risky is this batch considering quality and delays?

Condición	Points

critical_delay_prod = 1	+2

critical_quality = 1	+2

quality_flag = 1	+1


In [80]:
production["batch_risk_score"] = (
    production["critical_delay_prod"] * 2 +
    production["critical_quality"] * 2 +
    production["quality_flag"] * 1
)


In [81]:
production["batch_risk_score"].describe()


count    800.000000
mean       2.742500
std        1.493815
min        0.000000
25%        2.000000
50%        3.000000
75%        3.000000
max        5.000000
Name: batch_risk_score, dtype: float64

### KPI: Batch Risk Score
Composite score combining production delays and quality issues.
Used for prioritization and alerting.


### Feature Engineering Summary — Production

All KPIs and risk flags were validated and fall within expected
operational ranges. The dataset supports risk prioritization
and cost-quality tradeoff analysis.

---

### KPI — delivery_delay_days
How many days was the shipment actually delayed (or early)?

delivery_delay_days = actual_delivery - planned_delivery (in days)

"0 → arrived on the planned day"

"> 0 → arrived late"

"< 0 → arrived early (not bad, informational only)"

In [82]:
distribution["delivery_delay_days"] = (
    distribution["actual_delivery"] - distribution["planned_delivery"]
).dt.days


In [83]:
distribution["delivery_delay_days"].describe()


count    1000.000000
mean        2.207000
std         2.411216
min         0.000000
25%         0.000000
50%         1.000000
75%         4.000000
max        11.000000
Name: delivery_delay_days, dtype: float64

### KPI: Delivery Delay (Days)
Calculated as the difference between actual and planned delivery dates.


#### Flag — critical_delay_ship
Is the logistics delay operationally critical?

critical_delay_ship = 1 if delivery_delay_days >= 2

                      0 otherwise


In [84]:
distribution["critical_delay_ship"] = (
    (distribution["delivery_delay_days"] >= 2).astype(int)
)


In [85]:
distribution["critical_delay_ship"].value_counts()


critical_delay_ship
0    528
1    472
Name: count, dtype: int64

### Flag: Critical Shipping Delay
Identifies shipments delayed by two days or more.


#### Bandera — on_time_flag
Did this shipment arrive “on time” from an operational point of view?

on_time_flag = 1 if delivery_delay_days <= 1

               0 otherwise


In [86]:
distribution["on_time_flag"] = (
    (distribution["delivery_delay_days"] <= 1).astype(int)
)


In [87]:
distribution["on_time_flag"].value_counts()


on_time_flag
1    528
0    472
Name: count, dtype: int64

### Flag: On-Time Delivery
Indicates whether a shipment arrived within acceptable delivery time.


## Feature Engineering Summary — Distribution

Logistics KPIs and delay flags were calculated successfully.
These metrics enable on-time performance and delay risk analysis.


Delivery delay metrics and logistics performance flags were validated.

Distributions reflect a high-pressure operational context with

significant delays, supporting the urgent nature of the analysis.


---

## Why No Feature Engineering on dim_plants

The plants table is used as a pure dimension table.
It contains descriptive attributes (location, type, capacity)
and does not require derived metrics.

All operational KPIs are calculated at the fact level
to preserve analytical flexibility in Tableau.


## 4. Tableau Data Modeling

The data model follows a star-schema-inspired design using Tableau relationships.

- fact_batches contains production-level metrics (batch granularity)
- fact_shipments contains logistics-level metrics (shipment granularity)
- dim_plants provides descriptive attributes for manufacturing plants

Relationships were used instead of physical joins to avoid metric duplication
and ensure correct aggregation across different levels of detail.



## Tableau Calculations and Parameters

All business-critical KPIs were calculated during ETL.
Tableau is used exclusively for aggregation, filtering,
ranking, and scenario analysis.

Parameters were implemented to allow stakeholders
to adjust quality and delay thresholds without modifying
the ETL process.


LOAD — Final export

In [103]:
print("fact_batches:", production.shape)
print("fact_shipments:", distribution.shape)
print("dim_plants:", plants.shape)

print("\nColumns in fact_batches:")
print(production.columns)

print("\nColumns in fact_shipments:")
print(distribution.columns)

print("\nColumns in dim_plants:")
print(plants.columns)


fact_batches: (800, 17)
fact_shipments: (1000, 9)
dim_plants: (8, 4)

Columns in fact_batches:
Index(['batch_id', 'plant', 'country', 'product', 'production_date',
       'units_produced', 'units_rejected', 'production_time_hrs', 'delay_days',
       'production_cost', 'quality_issue', 'reject_rate', 'cost_per_unit',
       'quality_flag', 'critical_quality', 'critical_delay_prod',
       'batch_risk_score'],
      dtype='object')

Columns in fact_shipments:
Index(['batch_id', 'region', 'planned_delivery', 'actual_delivery',
       'transport_days', 'delivery_status', 'delivery_delay_days',
       'critical_delay_ship', 'on_time_flag'],
      dtype='object')

Columns in dim_plants:
Index(['plant', 'country', 'capacity_units_per_month', 'plant_type'], dtype='object')


In [89]:
# fact_batches
production["production_date"] = production["production_date"].dt.strftime("%Y-%m-%d")

# fact_shipments
distribution["planned_delivery"] = distribution["planned_delivery"].dt.strftime("%Y-%m-%d")
distribution["actual_delivery"] = distribution["actual_delivery"].dt.strftime("%Y-%m-%d")


In [104]:
# Output paths
batches_out = "../data/processed/fact_batches.csv"
shipments_out = "../data/processed/fact_shipments.csv"
plants_out = "../data/processed/dim_plants.csv"

# Export CSV in UTF-8
production.to_csv(batches_out, index=False, encoding="utf-8")
distribution.to_csv(shipments_out, index=False, encoding="utf-8")
plants.to_csv(plants_out, index=False, encoding="utf-8")

print("Files exported successfully:")
print(batches_out)
print(shipments_out)
print(plants_out)


Files exported successfully:
../data/processed/fact_batches.csv
../data/processed/fact_shipments.csv
../data/processed/dim_plants.csv


## Load Summary

Final datasets were exported as UTF-8 CSV files.
All tables are analytically ready and optimized
for consumption in Tableau using relationships.


# Final checks

In [91]:
total_batches = production.shape[0]
total_shipments = distribution.shape[0]

print("Total batches:", total_batches)
print("Total shipments:", total_shipments)


Total batches: 800
Total shipments: 1000


## Final Record Counts

- Total production batches: 800
- Total shipments: 1000


In [93]:
pct_critical_batches = (
    (production["batch_risk_score"] >= 3).mean()
)

print(f"% batches critical: {pct_critical_batches:.1%}")



% batches critical: 69.1%


## Production Risk Overview

- Percentage of critical batches (risk score ≥ 3): ~69.1%


In [94]:
pct_delayed_shipments = (
    (distribution["on_time_flag"] == 0).mean()
)

print(f"% delayed shipments: {pct_delayed_shipments:.1%}")


% delayed shipments: 47.2%


## Logistics Performance Overview

- Percentage of delayed shipments: ~47.2%


In [95]:
top5_reject_rate = (
    production
    .groupby("plant")["reject_rate"]
    .mean()
    .sort_values(ascending=False)
    .head(5)
)

top5_reject_rate


plant
Plant_F    0.197520
Plant_C    0.180245
Plant_E    0.096660
Plant_D    0.092856
Plant_A    0.088726
Name: reject_rate, dtype: float64

## Top 5 Plants by Average Reject Rate

Plants with the highest average rejection rates were identified,
indicating potential quality issues requiring attention.


In [96]:
top5_risk_score = (
    production
    .groupby("plant")["batch_risk_score"]
    .sum()
    .sort_values(ascending=False)
    .head(5)
)

top5_risk_score


plant
Plant_F    480
Plant_C    410
Plant_E    374
Plant_B    342
Plant_D    300
Name: batch_risk_score, dtype: int64

## Top 5 Plants by Total Risk Score

These plants concentrate the highest cumulative operational risk
across production batches.


In [ ]:

shipments_with_plant = distribution.merge(
    production[["batch_id", "plant"]],
    on="batch_id",
    how="left"
)

top5_delivery_delay = (
    shipments_with_plant
    .groupby("plant")["delivery_delay_days"]
    .mean()
    .sort_values(ascending=False)
    .head(5)
)

top5_delivery_delay


plant
Plant_D    2.373418
Plant_B    2.370787
Plant_C    2.308219
Plant_E    2.122995
Plant_A    2.074830
Name: delivery_delay_days, dtype: float64

## Top 5 Plants by Average Delivery Delay

Plants associated with the highest average delivery delays
were identified, highlighting logistics-related bottlenecks.


## ETL Closure Summary

Final validation confirms that:
- All datasets contain expected record volumes
- Production and logistics KPIs fall within realistic operational ranges
- A significant share of batches and shipments are classified as critical
- Risk and delays are concentrated in a limited number of plants

The data is analytically sound and ready for dashboarding and executive review.
